# 查询转换

查询转换是一组专注于重写和/或修改问题以进行检索的方法。

![Screenshot](./start.png)


In [ ]:
# 环境配置

! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


根据文档安装 `LangSmith` https://docs.smith.langchain.com/

In [ ]:
# langSmith 环境配置
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'your-api-key'


如果你是用 ` openAI` 作为 LLM，那么你需要在环境变量中设置 `OPENAI_API_KEY`

In [ ]:
os.environ['OPENAI_API_KEY'] = <your-api-key>


In [2]:
# 我这里是用 azure 的服务，所以这里我的代码是这样子的，你可以根据自己的情况进行修改
import os
import dotenv

# 加载.env 文件
dotenv.load_dotenv("../.env")


True

## Multi Query 多查询检索器

基于距离的向量数据库检索通过将查询嵌入（表示）在高维空间中，并根据“距离”找到相似的嵌入文档。然而，检索结果可能会因查询措辞的细微变化或嵌入未能很好地捕捉数据语义而有所不同。为了解决这些问题，有时需要进行提示工程/调整，但这可能会很繁琐。

MultiQueryRetriever 自动化了提示调整的过程，它使用大语言模型（LLM）从不同的角度为给定的用户输入查询生成多个查询。对于每个查询，它检索一组相关文档，并通过所有查询的唯一并集来获得更大范围的潜在相关文档集。通过为同一个问题生成多个角度的查询，MultiQueryRetriever 能够克服基于距离的检索的一些局限性，获取更加丰富的结果集。

流程图如下:

![Screenshot 2024-02-12 at 12.39.59 PM.png](./multi_query.png)

代码实现：

In [3]:
import bs4
import os
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 加载文档
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = WebBaseLoader(
    web_paths=("https://juejin.cn/post/7366149991159955466",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("article", "article-title")
        )
    ),
)
# 文档加载器加载文档
docs = loader.load()

# 文档拆分器
# from_tiktoken_encoder 使用tiktoken编码器的文本分割器来计长度。
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

# 文档拆分
splits = text_splitter.split_documents(docs)

# 实例化 embedding 模型 AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    model=os.environ.get("AZURE_EMBEDDING_TEXT_MODEL")
)

# 构建向量索引
vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

retriever = vectorstore.as_retriever()


### 构建 prompt template

In [17]:
from langchain.load import dumps, loads
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# 是用 prompt 指令让 LLM 构建 5 个不同纬度的问题，来让 LLM 从向量数据库中检索相关文档。
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

# template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
# Generate multiple search queries related to: {question} \n
# Output (4 queries):"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

# 初始化 LLM， LLM 的配置会自动获取环境变量中的值。
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME_GPT35"),
    temperature=0
)

generate_queries = (
    prompt_perspectives
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


def get_unique_union(documents: list[list]):
    """ 检索到文档的组合 """
    # 将列表展平，并将每个文档转换为字符串
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # 获取独特的文件
    unique_docs = list(set(flattened_docs))
    # 返回 langchain documents
    return [loads(doc) for doc in unique_docs]


question = "RAG优化技巧有哪些？"
result = generate_queries.invoke(question)
# 这里可以看到 LLM 返回的 5 个问题。
result


['1. 有哪些方法可以优化RAG？',
 '2. RAG的优化有哪些技巧可以使用？',
 '3. 如何提高RAG的性能？',
 '4. 有哪些策略可以改善RAG的效率？',
 '5. RAG优化方面有哪些值得尝试的方法？']

In [18]:
# 检索
question = "RAG优化技巧有哪些？"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
docs
# len(docs)


[Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='查询转换\n提高 RAG 系统效能的一个策略是添加一层查询理解层，也就是在实际进行检索前，先进行一系列的 Query Rewriting。具体而言，我们可以采用以下四种转换方法：\n1.1 路由：在不改变原始查询的基础上，识别并导向相关的工具子集，并将这些工具确定为处理该查询的首选。'),
 Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='RAG优化技巧|7大挑战与解決方式|提高你的LLM能力\n              \n    demo007x\n       \n                    2024-05-08\n                    \n                    63\n                   \n                    阅读13分钟\n                          RAG优化技巧|7大挑战与解決方式|提高你的LLM'),
 Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='1.调整检索策略\nLangchain中提供许多检索的方法，确保我们在RAG中能拿到最符合问题的文件，详细的列表可以参考官网，其中包含：'),
 Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='通过对 RAG 系统挑战的深入分析和优化，我们不仅可以提升LLM的准确性和可靠性，还能大幅提高用户对技术的信任度和满意度。\n希望这篇能帮助我们改善我们的 RAG 系统。\n推荐阅读：\n\n\n使用coze扣子搭建智能bot「程序员的工具箱」的思考和总结 - 掘金 (juejin.cn)\n\n\nRAG实操教程: langchai

LLm 返回的 5 个不同纬度的问题的答案。
这个流程首先将原始问题给到 LLM，然后 LLM 会返回 5 个不同纬度的问题，然后我们再将这 5 个问题给到 向量检索器，然后向量检索器会查找很多相关的答案，然后将这些 documents 进行唯一处理

queyr -> LLM -> 5 queries -> Vector Store -> 5 documents -> LLM -> Answer

构建最后的 chatbot chain 应用。

In [15]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG prompt
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# 构建最后的 RAG 链
final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

answer = final_rag_chain.invoke({"question": question})
answer


'RAG优化技巧包括添加一层查询理解层，进行一系列的Query Rewriting，以及采用四种转换方法：路由、数据清洗、信息压缩和对RAG系统挑战的深入分析和优化。'

## RAG-Fusion RAG 融合

![rag-fusion](./fusion.png)

RAG-Fusion 是一种搜索方法，旨在弥合传统搜索范式与人类查询多维度之间的差距。受检索增强生成（RAG）能力的启发，该项目更进一步，通过使用多重查询生成和互惠排名融合（Reciprocal Rank Fusion）对搜索结果进行重新排序。

RAG-Fusion有两个特点：

- 一个是问题增强，它会生成多个和原始问题类似的相关问题，提升检索的覆盖范围。举个例子，比如我想查询的是环境问题对我们的生活都会有什么影响，它可能就会帮你生成环境问题对我们的健康会有什么影响，环境问题对我们的经济会有什么，类似这样的相关问题。这样在向量搜索的时候，检索结果的覆盖面和多样性就更好了

- 另外一个是RAG-Fusion使用了RRF 倒数排序融合作为排序方法，相比普通排序，RRF更依赖于每个排序中的相对排名，更擅长组合来自不同策略的查询结果，比如使用 BM25 和 Embedding 做多路召回，RRF 提供的重排质量确实要更好一点

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion 这里要求 LLM 根据输入的问题产生多个问题
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# 构建查询链
generate_queries = (
    prompt_rag_fusion
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


In [20]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    """ 将多个排名文档列表和在RRF公式中使用的可选参数k进行融合。"""

    # 初始化一个字典，用于保存每个唯一文档的融合分数。
    fused_scores = {}

    # 遍历每个排名文档列表
    for docs in results:
        # 遍历列表中的每个文档，同时显示其排名（在列表中的位置）
        for rank, doc in enumerate(docs):
            # 将文档转换为字符串格式以用作密钥（假设文档可以序列化为 JSON）。
            doc_str = dumps(doc)
            # 如果文档尚未在fused_scores字典中，将其添加并设置初始分数为0。
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 如果有的话检索文档的当前分数
            previous_score = fused_scores[doc_str]
            # 使用 RRF 公式更新文档的分数：1 / (排名 + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # 根据它们的融合分数按降序对文档进行排序，以获取最终重新排名的结果。
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # 将重新排序的结果作为元组列表返回，每个元组包含文档和其融合得分。
    return reranked_results


retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
docs
# len(docs)


[(Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='然而，尽管LLM + RAG的能力已经让人惊叹，但我们在使用RAG优化LLM的过程中，还是会遇到许多挑战和困难，包括但不限于检索器返回不准确或不相关的数据，并且基于错误或过时信息生成答案。因此本文旨在提出RAG常见的7大挑战，并附带各自相应的优化方案，期望能够帮助我们改善RAG。'),
  0.06585580821434867),
 (Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='查询转换\n提高 RAG 系统效能的一个策略是添加一层查询理解层，也就是在实际进行检索前，先进行一系列的 Query Rewriting。具体而言，我们可以采用以下四种转换方法：\n1.1 路由：在不改变原始查询的基础上，识别并导向相关的工具子集，并将这些工具确定为处理该查询的首选。'),
  0.06558258417063283),
 (Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='通过对 RAG 系统挑战的深入分析和优化，我们不仅可以提升LLM的准确性和可靠性，还能大幅提高用户对技术的信任度和满意度。\n希望这篇能帮助我们改善我们的 RAG 系统。\n推荐阅读：\n\n\n使用coze扣子搭建智能bot「程序员的工具箱」的思考和总结 - 掘金 (juejin.cn)\n\n\nRAG实操教程: langchain+Milvus向量数据库创建你的本地知识库)'),
  0.048131080389144903),
 (Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='RAG优化技巧|7大挑战与解決方式|提高你的LLM能力\n              \n    demo007x\n       \n      

In [22]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

answer = final_rag_chain.invoke({"question": question})
answer


'RAG优化技巧包括添加一层查询理解层，进行一系列的Query Rewriting，路由，数据清洗和信息压缩。'

## 问题分解

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 构建 prompt，将一个问题分解为多个子问题
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

# 构建分解链
generate_queries_decomposition = (
    prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# 测试分解链
questions = generate_queries_decomposition.invoke({"question": question})
questions


['1. 什么是RAG优化技巧？', '2. RAG优化技巧的实际应用有哪些？', '3. 如何有效地实施RAG优化技巧？']

## Answer recursively  递归回答

![Screenshot 2024-02-18 at 1.55.32 PM.png](./answer_recursively.png)

可能有的人会问，已经有了 `RAG-Fusion`，为什么还有 `Answer recursively` 递归回答呢？

传统的文本生成方法通常涉及对模型的一次性查询，这可能会也可能不会产生所需的输出。迭代提示使我们能够以更可控的方式优化模型的输出。

`RAG-Fusion` 的思想是将多个检索器的结果作为输入传递给模型，以生成更准确和多样化的输出。

`Answer recursively`的思想是将模型的输出作为输入传递给模型，以生成更准确和多样化的输出。


In [24]:
# 构建 Answer recursively Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)


In [25]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser


def format_qa_pair(question, answer):
    """格式化问题和答案对"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

q_a_pairs = ""
for q in questions:
    rag_chain = (
        {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "q_a_pairs": itemgetter("q_a_pairs")}
        | decomposition_prompt
        | llm
        | StrOutputParser())

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n---\n" +  q_a_pair

answer


'根据提供的背景信息和相关问题+答案对，可以有效地实施RAG优化技巧的方法包括：\n\n1. 查询转换：通过添加查询理解层进行查询转换，识别并导向相关的工具子集，并将这些工具确定为处理查询的首选。\n2. 数据清洗：确保投入足够的精力去清洗数据，因为数据的质量直接影响到检索的效果。\n3. 信息压缩：采取信息压缩的方法来提高RAG系统的效能。\n\n这些方法旨在解决RAG系统面临的挑战和困难，以提高LLM的准确性和可靠性，以及提高用户对技术的信任度和满意度。因此，有效地实施RAG优化技巧需要在查询转换、数据清洗和信息压缩等方面进行综合考虑和实施。'

# Answer individually 把问题拆分

![Answer individually](./answer_individually.png)

`Answer individually` 分开回答的思想是，将问题拆分成多个小问题，然后分别回答这些小问题，最后将这些小问题的答案组合起来一起让 LLM 生成，形成最终的答案。这种思想可以帮助我们更好地理解问题，也可以帮助我们更好地回答问题。

In [48]:

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
# prompt_rag = hub.pull("rlm/rag-prompt")
prompt_template = """Answer the following question with chinese based on this context:

{context}

Question: {question}
"""
prompt_rag = ChatPromptTemplate.from_template(prompt_template)

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """
    对每个子问题进行检索和生成答案

    参数：
    question (str): 原始问题
    prompt_rag (PromptTemplate): 用于生成答案的提示模板
    sub_question_generator_chain (LLMChain): 用于生成子问题的链

    返回：
    rag_results (List[str]): 每个子问题的答案列表
    sub_questions (List[str]): 生成的所有子问题列表
    """
    # 使用我们的分解来生成子问题。
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    # 初始化一个列表，用于保存 RAG 链的结果。
    rag_results = []

    # 遍历每个子问题
    for sub_question in sub_questions:
        # 检索与每个子问题相关的文件
        retrieved_docs = retriever.invoke(sub_question)
        # 使用RAG链中检索到的文件和子问题。
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,"question": sub_question})
        print(answer)
        rag_results.append(answer)

    return rag_results, sub_questions


# 将检索和 RAG 过程封装在一个 RunnableLambda 中，以便集成到链中。
answers, questions = retrieve_and_rag(
    question, prompt_rag, generate_queries_decomposition)

# 这里生成 3 个问题以及对应的答案。
answers
print("----------")
questions


RAG优化技巧是指通过对RAG系统挑战的深入分析和优化，提升LLM的准确性和可靠性，以及提高用户对技术的信任度和满意度的方法和策略。
RAG优化技巧的实际应用包括添加查询理解层，在实际进行检索前进行一系列的查询重写，以及进行数据清洗和信息压缩等方法。
通过添加查询理解层，进行查询转换和数据清洗，以提高RAG系统的效能和准确性。同时，对RAG系统的挑战进行深入分析，并采取相应的优化方案，以提高用户对技术的信任度和满意度。
----------


['1. 什么是RAG优化技巧？', '2. RAG优化技巧的实际应用有哪些？', '3. 如何有效地实施RAG优化技巧？']

使用 qa_pairs 模式，将问题和答案作为一个整体进行检索。

In [49]:
def format_qa_pairs(questions, answers):
    """格式化问题和答案对，以生成格式化的字符串"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {
            question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()


context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context": context,"question":question})


'RAG优化技巧包括通过对RAG系统挑战的深入分析和优化，提升LLM的准确性和可靠性，以及提高用户对技术的信任度和满意度的方法和策略。实际应用包括添加查询理解层，在实际进行检索前进行一系列的查询重写，以及进行数据清洗和信息压缩等方法。有效地实施RAG优化技巧需要通过添加查询理解层，进行查询转换和数据清洗，以提高RAG系统的效能和准确性，并对RAG系统的挑战进行深入分析，并采取相应的优化方案，以提高用户对技术的信任度和满意度。'

## Part 8: Step Back 向后退

![Screenshot 2024-02-12 at 1.14.43 PM.png](./step_back.png)

`Step Back Prompting` 通过将复杂任务分解为抽象和推理两个步骤，帮助语言模型更好地处理包含大量细节的任务，提高其在复杂推理任务中的性能。

In [50]:
# 示例
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# 我们现在将这些转换为示例消息。
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # 少样本提示词
        few_shot_prompt,
        ("user", "{question}"),
    ]
)


FewShotChatMessagePromptTemplate() 支持少量示例的聊天提示模板。

该提示模板生成的高级结构是由前缀消息、示例消息和后缀消息组成的消息列表。

这种结构使得创建具有中间示例对话变得可能，例如：

系统：你是一个乐于助人的AI助手
用户：2+2等于多少？
AI：4
用户：2+3等于多少？
AI：5
用户：4+4等于多少？

这个提示模板可以用来生成固定列表的示例，也可以根据输入动态选择示例。

In [51]:
generate_queries_step_back = prompt | llm | StrOutputParser()
answer = generate_queries_step_back.invoke({"question": question})
answer


'What are some techniques for optimizing RAG?'

In [54]:
# 响应提示
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # 使用正常问题获取上下文
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # 使用回溯问题来获取上下文
        "step_back_context": generate_queries_step_back | retriever,
        # 把问题传给 LLM 回答
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke({"question": question})
answer


'RAG优化技巧包括但不限于以下几种方法：\n\n1. 查询转换：添加一层查询理解层，即在实际进行检索前，先进行一系列的Query Rewriting。具体而言，可以采用路由、识别并导向相关的工具子集，并将这些工具确定为处理该查询的首选等转换方法。\n\n2. 数据清洗：数据的质量直接影响到检索的效果，因此在优化RAG系统时，需要确保已经投入足够的精力去清洗数据，以确保数据的质量和准确性。\n\n3. 信息压缩：在优化RAG系统时，可以考虑对信息进行压缩，以减少数据量和提高检索效率。\n\n4. 深入分析和优化：通过对RAG系统挑战的深入分析和优化，可以提升LLM的准确性和可靠性，同时提高用户对技术的信任度和满意度。\n\n5. 添加查询理解层：在实际进行检索前，先进行一系列的Query Rewriting，以提高RAG系统的效能和准确性。\n\n这些优化技巧可以帮助改善RAG系统的性能和效果，提高LLM的能力和可靠性。'

## HyDE

![Screenshot 2024-02-12 at 1.12.45 PM.png](./hyde.png)


HyDE（Hypothetical Document Embeddings）是一种用于零样本密集检索的方法，其工作原理如下：

- 分解任务：将密集检索分解为两个任务，一个是由指令跟随语言模型执行的生成任务，另一个是由对比编码器执行的文档 - 文档相似性任务。
- 生成假设文档：对于给定的查询，首先将其输入到指令跟随语言模型（如 InstructGPT）中，并指示它 “编写一个回答问题的文档”，即生成一个假设文档。该文档虽然不是真实的，可能包含事实错误，但期望它能够捕获相关性模式。
- 编码假设文档：使用无监督对比编码器（如 Contriever）将假设文档编码为嵌入向量。在这里，期望编码器的密集瓶颈起到有损压缩机的作用，过滤掉嵌入向量中多余的（幻觉）细节。
- 检索相似文档：使用这个向量在语料库嵌入中进行搜索，根据向量相似性检索最相似的真实文档并返回。检索利用了在对比训练期间编码在内积中的文档 - 文档相似性。

In [55]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# HyDE文档生成
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser()
)

answer = generate_docs_for_retrieval.invoke({"question": question})
answer


'RAG（Random Access Generator）是一种用于生成随机访问数据的技术，它可以用于优化数据访问的效率和性能。RAG优化技巧包括但不限于以下几点：\n\n1. 数据分布优化：通过对数据进行合理的分布和组织，可以减少数据访问的随机性，提高数据访问的效率。例如，可以将数据按照访问频率进行分组，将频繁访问的数据放置在更容易访问的位置，从而减少访问时间。\n\n2. 缓存优化：利用缓存技术可以减少数据访问的次数，提高数据访问的速度。RAG可以通过合理设置缓存大小和缓存策略来优化数据访问性能。\n\n3. 并行访问优化：利用并行访问技术可以同时处理多个数据访问请求，提高数据访问的并发性能。RAG可以通过合理设计并行访问的机制来优化数据访问的效率。\n\n4. 数据预取优化：通过预取数据可以减少数据访问的延迟，提高数据访问的速度。RAG可以通过预取数据的策略来优化数据访问性能。\n\n总之，RAG优化技巧可以通过合理的数据分布、缓存、并行访问和数据预取等手段来提高数据访问的效率和性能。这些技巧可以根据具体的应用场景和需求进行灵活的调整和优化，从而实现更高效的数据访问。'

In [56]:
# 检索
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question": question})
retireved_docs


[Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='然而，尽管LLM + RAG的能力已经让人惊叹，但我们在使用RAG优化LLM的过程中，还是会遇到许多挑战和困难，包括但不限于检索器返回不准确或不相关的数据，并且基于错误或过时信息生成答案。因此本文旨在提出RAG常见的7大挑战，并附带各自相应的优化方案，期望能够帮助我们改善RAG。'),
 Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='查询转换\n提高 RAG 系统效能的一个策略是添加一层查询理解层，也就是在实际进行检索前，先进行一系列的 Query Rewriting。具体而言，我们可以采用以下四种转换方法：\n1.1 路由：在不改变原始查询的基础上，识别并导向相关的工具子集，并将这些工具确定为处理该查询的首选。'),
 Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='1. 数据清洗\n数据的质量直接影响到检索的效果，这个痛点再次突显了优质数据的重要性。在责备你的 RAG 系统之前，请确保你已经投入足够的精力去清洗数据。\n2. 信息压缩'),
 Document(metadata={'source': 'https://juejin.cn/post/7366149991159955466'}, page_content='Auto Merging Retriever\nMetadata Replacement + Node Sentence Window\nRecursive Retriever\n\n总结\n本文探讨了使用 RAG 技术时可能面临的七大挑战，并针对每个挑战提出了具体的优化方案，以提升系统准确性和用户体验。')]

In [59]:
# 构建 RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

answer= final_rag_chain.invoke({"context": retireved_docs,"question":question})
answer


'RAG优化技巧包括添加查询理解层进行查询转换，数据清洗和信息压缩。'